# GkNN for 1d Arbitrary Grid

In [ ]:
%config InlineBackend.figure_format = 'svg'
import random
import torch
import os
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
from timeit import default_timer
from scipy.io import loadmat


sys.path.append('../')
from models import FNN1d, FNN_train, construct_model, compute_1dFourier_bases_arbitrary,compute_1dWeights
torch.set_printoptions(precision=16)

seed = 0


os.environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.set_printoptions(precision=16)


## Data Loading

In [ ]:
data_path = "../mytest/data/burgers_data_R10.mat"
data = loadmat(data_path)
data_in = data["a"]
data_out = data["u"]

L = 1
subrate = 2**3
gridsize = 2**13 // subrate
x_data = torch.from_numpy(data["a"][:, ::subrate]).unsqueeze(-1)
y_data = torch.from_numpy(data["u"][:, ::subrate])
num_data = x_data.size(0)

grid = torch.linspace(0, L, gridsize + 1)[:-1].unsqueeze(-1)
x_data = torch.cat((x_data, grid.repeat([num_data, 1, 1])), dim=-1)

print("number of data:", num_data, ", grid size now:", gridsize)

## Generation of Arbitrary Grid

In [ ]:
s = 2**8
np.random.seed(seed)
index_selected = np.random.choice(range(gridsize), s, replace=False)

index_selected.sort()
index_selected[0], index_selected[s - 1] = 0, gridsize - 1
grid = torch.tensor(index_selected) / gridsize
print("points selected:", s)
print("index selected:\n", index_selected)

# selecting points, concatenating x and grid
grid_expanded=grid.unsqueeze(0).unsqueeze(-1)
x_selected = x_data[:, index_selected, :]
y_selected = y_data[:, index_selected]

plt.figure(figsize=(10, 6))
plt.scatter(grid, y_selected[0, :], marker=".")

## Selecting Data for Training and Testing

In [ ]:
ntrain = 2**10
ntest = 2**9

x_train = x_selected[:ntrain, :, :].float()
y_train = y_selected[:ntrain, :].float()
x_test = x_selected[:-ntest, :, :].float()
y_test = y_selected[:-ntest, :].float()
print(x_train.size())

## Generation of Basis
by SVD
$$USV^T=Y, and~ U^TU=I_{n\times n}$$
let 
$$W:=diag\{\Delta x_1,\Delta x_2,\cdots,\Delta x_n\},B:=\sqrt{W}^{-1}U$$
then
$$B^TWB=U^T\sqrt{W}^{-T}W\sqrt{W}^{-1}U=U^TU=I_{n\times n}$$


In [ ]:
k_max = 33
weights = compute_1dWeights(grid)

bases_fourier, wbases_fourier = compute_1dFourier_bases_arbitrary(
    s, k_max, grid, weights
)

pca_data = y_train.T
U, S, VT = np.linalg.svd(pca_data)
U = torch.from_numpy(U.astype(np.float32))
bases_pca = U[:, :k_max] / torch.sqrt(weights.unsqueeze(1))
wbases_pca = U[:, :k_max] * torch.sqrt(weights.unsqueeze(1))

plt.figure(figsize=(10, 6))
for i in range(1, 4):
    plt.scatter(grid, bases_fourier[:, i], marker=".", label="base" + str(i + 1))
plt.legend()
print(
    f"verify orthonormality in fourier bases:\n Int(base3^2 dx)={sum(bases_fourier[:, 6] * wbases_fourier[:, 6]).item()},Int(base3*base5 dx)={sum(bases_fourier[:, 1] * wbases_fourier[:, 10]).item()}"
)

plt.figure(figsize=(10, 6))
plt.scatter(grid, pca_data[:, 0], marker=".", label="u")
for i in range(3):
    plt.scatter(grid, bases_pca[:, i], marker=".", label="base" + str(i + 1))
plt.legend()
print(
    f"verify orthonormality in pca bases:\n Int(base3^2 dx)={sum(bases_pca[:, 3] * wbases_pca[:, 3]).item()},Int(base3*base5 dx)={sum(bases_pca[:, 3] * wbases_pca[:, 5]).item()}"
)

### Configures

In [10]:
model_type = "GalerkinNO"
galerkin_config_std1 = {
    "type": "GalerkinConv",
    "num_modes": k_max,
    "bases": bases_fourier,
    "wbases": wbases_fourier,
}
galerkin_config_std2 = {
    "type": "GalerkinConv",
    "num_modes": k_max,
    "bases": bases_pca,
    "wbases": wbases_pca,
}
attention_config_std = {
    "type": "Attention",
    "num_heads": 1,
    "attention_type": "galerkin",
}
layer_configs = [galerkin_config_std1, galerkin_config_std1, galerkin_config_std2]


config = {
    "model": {
        "model": model_type,
        "dim": 1,
        "fc_dim": 128,
        "layers": [64] * 4,
        "in_dim": 2,
        "out_dim": 1,
        "act": "gelu",
        "pad_ratio": -1,
        "layer_configs": layer_configs,
    },
    "train": {
        "base_lr": 0.001,
        "weight_decay": 1.0e-4,
        "epochs": 200,
        "scheduler": "MultiStepLR",
        "milestones": [50, 100, 150],
        "scheduler_gamma": 0.5,
        "batch_size": 8,
        "normalization_x": True,
        "normalization_y": True,
        "normalization_dim": [],
    },
}


model = construct_model(config)
print(model)

GkNN(
  (fc0): Linear(in_features=2, out_features=64, bias=True)
  (sp_layers): ModuleList(
    (0-2): 3 x GalerkinConv()
  )
  (ws): ModuleList(
    (0-2): 3 x Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  )
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


### Training

In [11]:
print("Start training ", config["model"]["model"])
train_rel_l2_losses, test_rel_l2_losses, test_l2_losses = FNN_train(
    x_train,
    y_train,
    x_test,
    y_test,
    config,
    model,
    save_model_name="../models/save/test.pth",
)

Start training  GalerkinNO
Epoch :  0  Rel. Train L2 Loss :  0.32311135379131883  Rel. Test L2 Loss :  0.21017331435965994  Test L2 Loss :  0.09608563875857119
Epoch :  1  Rel. Train L2 Loss :  0.14067617885302752  Rel. Test L2 Loss :  0.10468750939859699  Test L2 Loss :  0.04599779465934262
Epoch :  2  Rel. Train L2 Loss :  0.07661074574571103  Rel. Test L2 Loss :  0.06700280324245493  Test L2 Loss :  0.030024038424016908
Epoch :  3  Rel. Train L2 Loss :  0.06340742399333976  Rel. Test L2 Loss :  0.062405989485948034  Test L2 Loss :  0.02886938923135555
Epoch :  4  Rel. Train L2 Loss :  0.05277043324895203  Rel. Test L2 Loss :  0.05424540881843617  Test L2 Loss :  0.025208435457898304
Epoch :  5  Rel. Train L2 Loss :  0.04785550673841499  Rel. Test L2 Loss :  0.04712289554299787  Test L2 Loss :  0.021518290086532943
Epoch :  6  Rel. Train L2 Loss :  0.041403880168218166  Rel. Test L2 Loss :  0.05020312602088476  Test L2 Loss :  0.022696815722156316
Epoch :  7  Rel. Train L2 Loss :  0.